In [1]:
%matplotlib inline
import os
import time
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.lines as mlines
from sklearn.utils import shuffle
from scipy.stats import ortho_group
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from xnn.sosxnn import SOSxNN

In [2]:
# Simulation
corr = 0.5
noise_sigma = 1
dummy_num = 0
feature_num = 10
test_num = 10000
data_num = 10000

proj_matrix = np.zeros((feature_num,4))
proj_matrix[:7, 0] = np.array([1,0,0,0,0,0,0])
proj_matrix[:7, 1] = np.array([0,1,0,0,0,0,0])
proj_matrix[:7, 2] = np.array([0,0,0.5,0.5,0,0,0])
proj_matrix[:7, 3] = np.array([0,0,0,0,0.2,0.3,0.5])

def data_generator1(data_num, feature_num, corr, proj_matrix, noise_sigma):
    u = np.random.uniform(-1,1, [data_num, 1])
    t= np.sqrt(corr/(1-corr))
    X = np.zeros((data_num, feature_num))
    for i in range(feature_num):
        X[:, i:i+1] = (np.random.uniform(-1,1,[data_num,1])+t*u)/(1+t)
    Y = np.reshape(2*np.dot(X, proj_matrix[:,0])+0.2*np.exp(-4*np.dot(X, proj_matrix[:,1])) + \
              3*(np.dot(X, proj_matrix[:,2]))**2+2.5*np.sin(np.pi*np.dot(X, proj_matrix[:,3])), [-1,1]) + \
              noise_sigma*np.random.normal(0,1, [data_num,1])
    return X, Y

np.random.seed(0)
X, Y = data_generator1(data_num+test_num, feature_num+dummy_num, corr, proj_matrix, noise_sigma)
scaler_x = MinMaxScaler((-1, 1)); scaler_y = MinMaxScaler((-1, 1))
sX = scaler_x.fit_transform(X); sY = scaler_y.fit_transform(Y)
train_x, test_x, train_y, test_y = train_test_split(sX, sY, test_size = test_num)

In [ ]:
tf.random.set_seed(1)
np.random.seed(1)
model = SOSxNN(input_num=10, input_dummy_num=0, subnet_num=10, subnet_arch=[10, 6], task="Regression",
               activation_func=tf.tanh, batch_size=1000, training_epochs=5000, lr_bp=0.001, lr_cl=0.1,
               beta_threshold=0.01, tuning_epochs=0, l1_proj=0.001, l1_subnet = 0.01, smooth_lambda=10**(-5),
               verbose=True, val_ratio=0.2, early_stop_thres=500)
model.fit(train_x, train_y)
model.visualize("./", "test", train_x)

tr_pred = model.predict(model.tr_x) 
val_pred = model.predict(model.val_x) 
pred_test = model.predict(test_x)
sosxnn_mse_stat = np.hstack([np.round(np.mean((scaler_y.inverse_transform(tr_pred) - scaler_y.inverse_transform(model.tr_y))**2),5),\
                             np.round(np.mean((scaler_y.inverse_transform(val_pred) - scaler_y.inverse_transform(model.val_y))**2),5),\
                             np.round(np.mean((scaler_y.inverse_transform(pred_test) - scaler_y.inverse_transform(test_y))**2),5)])
print(sosxnn_mse_stat)

Initial training.
Training epoch: 1, train loss: 0.05350, val loss: 0.05221
Training epoch: 2, train loss: 0.04682, val loss: 0.04430
Training epoch: 3, train loss: 0.04163, val loss: 0.04032
Training epoch: 4, train loss: 0.03716, val loss: 0.03612
Training epoch: 5, train loss: 0.03404, val loss: 0.03336
Training epoch: 6, train loss: 0.03166, val loss: 0.03108
Training epoch: 7, train loss: 0.02991, val loss: 0.02982
Training epoch: 8, train loss: 0.02873, val loss: 0.02871
Training epoch: 9, train loss: 0.02788, val loss: 0.02783
Training epoch: 10, train loss: 0.02725, val loss: 0.02716
Training epoch: 11, train loss: 0.02653, val loss: 0.02666
Training epoch: 12, train loss: 0.02607, val loss: 0.02589
Training epoch: 13, train loss: 0.02575, val loss: 0.02559
Training epoch: 14, train loss: 0.02553, val loss: 0.02541
Training epoch: 15, train loss: 0.02539, val loss: 0.02531
Training epoch: 16, train loss: 0.02524, val loss: 0.02533
Training epoch: 17, train loss: 0.02527, val lo

Training epoch: 140, train loss: 0.02039, val loss: 0.02071
Training epoch: 141, train loss: 0.02023, val loss: 0.02060
Training epoch: 142, train loss: 0.02015, val loss: 0.02058
Training epoch: 143, train loss: 0.02014, val loss: 0.02060
Training epoch: 144, train loss: 0.02008, val loss: 0.02052
Training epoch: 145, train loss: 0.02002, val loss: 0.02046
Training epoch: 146, train loss: 0.02029, val loss: 0.02056
Training epoch: 147, train loss: 0.02013, val loss: 0.02046
Training epoch: 148, train loss: 0.02023, val loss: 0.02056
Training epoch: 149, train loss: 0.02006, val loss: 0.02052
Training epoch: 150, train loss: 0.02007, val loss: 0.02053
Training epoch: 151, train loss: 0.02052, val loss: 0.02112
Training epoch: 152, train loss: 0.02010, val loss: 0.02061
Training epoch: 153, train loss: 0.01993, val loss: 0.02034
Training epoch: 154, train loss: 0.02000, val loss: 0.02032
Training epoch: 155, train loss: 0.02000, val loss: 0.02033
Training epoch: 156, train loss: 0.01996

In [ ]:
tf.random.set_seed(1)
np.random.seed(1)
model = SOSxNN(input_num=10, input_dummy_num=0, subnet_num=10, subnet_arch=[10, 6], task="Regression",
               activation_func=tf.tanh, batch_size=1000, training_epochs=5000, lr_bp=0.001, lr_cl=0.1,
               beta_threshold=0.01, tuning_epochs=0, l1_proj=0.001, l1_subnet = 0.01, smooth_lambda=10**(-4),
               verbose=True, val_ratio=0.2, early_stop_thres=500)
model.fit(train_x, train_y)
model.visualize("./", "test", train_x)

tr_pred = model.predict(model.tr_x) 
val_pred = model.predict(model.val_x) 
pred_test = model.predict(test_x)
sosxnn_mse_stat = np.hstack([np.round(np.mean((scaler_y.inverse_transform(tr_pred) - scaler_y.inverse_transform(model.tr_y))**2),5),\
                             np.round(np.mean((scaler_y.inverse_transform(val_pred) - scaler_y.inverse_transform(model.val_y))**2),5),\
                             np.round(np.mean((scaler_y.inverse_transform(pred_test) - scaler_y.inverse_transform(test_y))**2),5)])
print(sosxnn_mse_stat)

In [ ]:
tf.random.set_seed(1)
np.random.seed(1)
model = SOSxNN(input_num=10, input_dummy_num=0, subnet_num=10, subnet_arch=[10, 6], task="Regression",
               activation_func=tf.tanh, batch_size=1000, training_epochs=5000, lr_bp=0.001, lr_cl=0.1,
               beta_threshold=0.01, tuning_epochs=100, l1_proj=0.001, l1_subnet = 0.01, smooth_lambda=10**(-3),
               verbose=True, val_ratio=0.2, early_stop_thres=500)
model.fit(train_x, train_y)
model.visualize("./", "test", train_x)

tr_pred = model.predict(model.tr_x) 
val_pred = model.predict(model.val_x) 
pred_test = model.predict(test_x)

sosxnn_mse_stat = np.hstack([np.round(np.mean((scaler_y.inverse_transform(tr_pred) - scaler_y.inverse_transform(model.tr_y))**2),5),\
                             np.round(np.mean((scaler_y.inverse_transform(val_pred) - scaler_y.inverse_transform(model.val_y))**2),5),\
               np.round(np.mean((scaler_y.inverse_transform(pred_test) - scaler_y.inverse_transform(test_y))**2),5)])
print(sosxnn_mse_stat)